How to turn this notebook into slides? Run:
```
$ ipython nbconvert --to slides lecture.ipynb  --post serve
```
OR (better), install the RISE Jupyter extension:
```
conda install -c damianavila82 rise
```
and hit the 'Slideshow' toolbar button.

This notebook uses the [python-markdown](http://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/python-markdown/readme.html) extension, allowing it to run code inline with Markdown text.

To install this extension, and the [configurator GUI](https://github.com/Jupyter-contrib/jupyter_nbextensions_configurator), while Jupyter is ***not*** running run:
```bash
conda install -c defaults -c conda-forge jupyter_contrib_nbextensions
conda install -c defaults -c conda-forge jupyter_nbextensions_configurator
```
and then enable python-markdown in the `Nbextensions` tab that will show up next time you start Jupyter.

To have the markup in all cells show up in this notebook, you need to mark it as 'Trusted'. The notebook will be marked as trusted once you re-evaluate all cells.

## Reading for Thursday

On Thursday we're going to talk more about Python dicts, list comprehensions, modules and object oriented programming. **Please skim through these in time for the next class** (it should take you ~2-3 hours):

* List comprehensions:
  * http://www.secnetix.de/olli/Python/list_comprehensions.hawk
  * http://www.pythonforbeginners.com/basics/list-comprehensions-in-python
  * http://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/
  * Learning Python, Chapter 14


* Dictionaries:
  * https://www.python-course.eu/dictionaries.php
  * http://introtopython.org/dictionaries.html
  * Learning Python, Chapter 8


* Modules:
  * https://docs.python.org/3/tutorial/modules.html
  * Learning Python, Chapter 22
  * Learning Python, Chapter 23 (through "Module Usage”).


* Python Classes/Objects:
  * https://jeffknupp.com/blog/2014/06/18/improve-your-python-python-classes-and-object-oriented-programming/ (you can stop at 'Inheritance' for now)
  * Learning Python, Chapter 26


Some helper functions to make it easier to show code listings.

In [6]:
def show_code_listing(fn):
    from IPython.display import display, Markdown
    return Markdown( '```python\n{}\n```'.format(open(fn).read()))

def execute_and_show(cmdline):
    from IPython.display import display, Markdown
    res = ! $cmdline
    res = '\n'.join(['    ' + line for line in res])
    print('```bash\n    ${}\n{}\n```'.format(cmdline, res))

## Problem 4: Making the code user friendly

When we enter an argument that's not a number:
{{ execute_and_show('./prog3.py 2 x') }}
It would be nicer to check for the number and type of arguments, and get a user-friendly error message if either are incorrect:
{{ execute_and_show('./prog4.py 2 x') }}
{{ execute_and_show('./prog4.py 2') }}

## Implementation v1

In [7]:
show_code_listing("prog4.py")

```python
#!/usr/bin/env python

import sys
import re

isNumber = r'^[1-9][0-9]*\.?[0-9]*([Ee][+-]?[0-9]+)?$';

if len(sys.argv) != 3:
    print("Error: two arguments required.")
    print("")
    print("Usage: {} <arg1> <arg2>".format(sys.argv[0]))
    exit(0)

if not re.search(isNumber, sys.argv[1]):
    print("Error: the first argument must be a number")
    print("")
    print("Usage: {} <arg1> <arg2>".format(sys.argv[0]))
    exit(0)

if not re.search(isNumber, sys.argv[2]):
    print("Error: the second argument must be a number")
    print("")
    print("Usage: {} <arg1> <arg2>".format(sys.argv[0]))
    exit(0)

a = float(sys.argv[1])
b = float(sys.argv[2])

print (a + b)

```

Discussion:
* We encounter our first [flow control](https://docs.python.org/3/tutorial/controlflow.html) statement -- `if`. It allows us to choose which path to take based on the result of evaluating a _condition_.
* I checked whether there are two arguments by using the `len()` built-in function that returns the length of the list. Note that the `sys.argv` list includes the name of the program; thus testing for `3` instead of `2`.
* I used [Python regular expressions](https://docs.python.org/3.4/library/re.html) to check for the format of each string, before attempting to convert them to a floating point number.

### Group Work:

My implementation above has lots of repetition (e.g., the same usage string is printed out three times). We know that functions are a way to avoid excessive repetition. Python functions have the following form:

```python
def func_name(a, b, c, d):
    ... function body ...
    return some_value
```

Let's change my implementation so it minimizes repetition by using functions. Identify repetitive parts and [refactor](https://en.wikipedia.org/wiki/Code_refactoring) them into functions.

## Implementation v2

In [8]:
show_code_listing("prog5.py")

```python
#!/usr/bin/env python

import sys
import re

def exit_with_usage():
    """ Prints a usage message and exits the program. """
    print("")
    print("Usage: {} <arg1> <arg2>".format(sys.argv[0]))
    exit(0)

def number_or_exit(s, msg):
    """ Test if argument s is a number, abort execution otherwise. """
    isNumber = r'^[1-9][0-9]*\.?[0-9]*([Ee][+-]?[0-9]+)?$';

    if not re.search(isNumber, s):
        print("Error: the {} argument must be a number".format(msg))
        exit_with_usage()

#### The main program begins here

if len(sys.argv) != 3:
    print("Error: two arguments required.")
    exit_with_usage()

number_or_exit(sys.argv[1], 'first')
number_or_exit(sys.argv[2], 'second')

a = float(sys.argv[1])
b = float(sys.argv[2])

print (a + b)

```

## Problem 5: A Better Way to Deal with Errors -- Exceptions

Note how most of our code now is just error checking and handling. The actual purpose of our code is completely hidden by all the error checking code!

This doesn't have to be the case. In Python, it is possible to elegantly separate the error checking code from the main algorithm through the use of [exceptions](https://docs.python.org/3/tutorial/errors.html#exceptions). The big idea is that code can _raise an exception_ when some assumption or rule is violated.

Example:

In [9]:
def div(a, b):
    if b == 0:
        raise ZeroDivisionError("Trying to divide by zero!")

    return a / b

print("Division result: {}".format(div(1, 2)))
print("Division result: {}".format(div(1, 0)))

Division result: 0


ZeroDivisionError: Trying to divide by zero!

When an exception is raised, Python stops executing the program and returns immediately from the called function. If an exception is not _caught_, it will interupt the program and print out a _stack trace_ (the output above) that gives you an idea of where an exception occured.

You've encountered this already:
{{ execute_and_show('./prog3.py 2 x') }}
The output above is the result of an _uncaught exception_.

The idea is that you can "catch" these exceptions, and handle the error condition gracefully (i.e., by writing out a nice error message). This is accomplished through the use of a "`try ... except`" clause:

In [10]:
def div(a, b):
    if b == 0:
        raise ZeroDivisionError("Trying to divide by zero!")

    return a / b

try:
    print("Division result: {}".format(div(1, 2)))
    print("Division result: {}".format(div(1, 0)))
    print("Division result: {}".format(div(1, 3)))
except ZeroDivisionError as err:
    print(err)
    
print("I'm not dead!")

Division result: 0
Trying to divide by zero!
I'm not dead!


What's happened:
* We've now wrapped our calls to `div()` in a `try...except` block.
* When an exception was raised, the program immediately jumped to the first `except` block that matched the type of the raised exception. We say that an exception has been _handled_.
* After that, execution continued (note how it printed the message about not being dead)
* Finally, note how `div(1, 3)` line has not been invoked -- because an exception has been raised, the execution continued in the `except` block. Python does not "go back" after handling the exception!

### Exceptions are the "Pythonic Way"

This is the preferred way to signal and check for errors in Python. Virtually all errors ("exceptional situations") are reported by raising an exception.

Actually, we didn't need to explicitly check for `b=0` -- Python itself raises an exception if we try to divide by zero:

In [11]:
1 / 0

ZeroDivisionError: integer division or modulo by zero

or another example, indexing past the end of a list:

In [12]:
a = ["a", "b", "c"]
a[4]

IndexError: list index out of range

We can have multiple `except` blocks, to handle different kinds of exceptions:

In [13]:
arr = [0, 1, 2, 3]
i, j = 1, 5
try:

    print(arr[i] / arr[j])

except ZeroDivisionError as err:
    print(err)
except IndexError as err:
    print(err)
    
print("moving on!")

list index out of range
moving on!


Compare that to:

In [14]:
arr = [0, 1, 2, 3]
i, j = 1, 5

if i < 0 or i >= len(arr) or j < 0 or j >= len(arr):
    print("list index out of range")
elif arr[j] == 0:
    print("division by zero")
else:
    print(arr[i] / arr[j])

print("moving on!")

list index out of range
moving on!


Note:
* The code using exceptions is _cleaner_ -- it neatly separates the part that does work (divides two numbers) from the part that handles exceptional situations.
* The code using exceptions is _shorter_ -- shorter coe, fewer bugs.
* The philosophical approach to errors is fundamentally different:
  1. the implementation with `if` statements tries to _avoid_ even trying to break the rules (dividing by zero, indexing out of bounds, ...)
  1. the implementation with exceptions detects when the rules were attempted to be broken and reacts to that situation.

## It's Easier to Ask for Forgiveness than Permission

What we've done above is an example of the [***EAFP***](https://docs.python.org/3.6/glossary.html) coding style:

>   It's **E**asier to **A**sk for **F**orgiveness, than **P**ermission.
>

that is core to well-written, readable, Python code.

How do you know what exceptions you could (should) be catching?
* The documentation of (typically) tells you which exceptions any given function may raise.
* Look at Python's [list of built-in exceptions](https://docs.python.org/3/library/exceptions.html)
* Try it!

## Group Work

Now back to our program: let's change it so that instead of the many if statements, we use exceptions.

## Implementation

In [15]:
show_code_listing("prog8.py")

IOError: [Errno 2] No such file or directory: 'prog8.py'

Discusion:
* This is _much_ cleaner now! Just a quick glance gives you an idea what this code does!
* Exceptions are "caught" in a [`try-except` block](https://docs.python.org/3/tutorial/errors.html#handling-exceptions).
* There was no need for the `number_or_exit()` function any more, or the messy loop, etc.!
* The code is now much more [_Pythonic_](http://docs.python-guide.org/en/latest/writing/style/).

Remember:

>   It's **E**asier to **A**sk for **F**orgiveness, than **P**ermission.
>

This is the Pythonic Way!

## Problem 6: Product of all arguments

Now let's extend the program to compute the _product_ of all arguments given on the command line. Example:
{{ execute_and_show('./prog9.py 1.5 2.5 3.5 4.5') }}

## Implementation

In [16]:
show_code_listing("prog9.py")

```python
#!/usr/bin/env python

import sys

def exit_with_msg(msg):
    """ Prints a usage message and exits the program. """
    print("{}\n\nUsage: {} <arg1> <arg2>".format(msg, sys.argv[0]))
    exit(0)

#### The main program begins here

try:
    prod = 1
    for arg in sys.argv[1:]:
        prod *= float(arg)
    print(prod)
except ValueError as e:
    exit_with_msg("Error: {}. All arguments must be numbers.".format(e))

```

Discusion:
* The change was pretty straightforward; just a `for`-loop over all arguments
* No need to catch the `IndexError` exception any more

## Implementation v2

Let's take a slightly different approach. Note we can view this as a two-part problem:

1. Converting the list of command line arguments from strings to floats
1. Summing up the resulting list.

So another solution could be:

In [17]:
show_code_listing("prog10.py")

```python
#!/usr/bin/env python

import sys

def exit_with_msg(msg):
    """ Prints a usage message and exits the program. """
    print("{}\n\nUsage: {} <arg1> <arg2>".format(msg, sys.argv[0]))
    exit(0)

def to_floats(lst):
    """ Returns a list of strings corresponding to a list of floats """
    vals = []
    for arg in lst:
        vals.append( float(arg) )
    return vals

def prod(lst):
    """ Computes and returns the product of a list """
    res = 1.
    for x in lst: res *= x
    return res

#### The main program begins here

try:
    float_list = to_floats( sys.argv[1:] )
    print ( prod( float_list ) )
except ValueError as e:
    exit_with_msg("Error: {}. All arguments must be numbers.".format(e))

```

Discusion:
* We've created two functions: `to_floats`, to solve the conversion, and `prod`, to compute the product.
* In `to_floats()`, we used the `append` function to add new items to the list.
* In many ways, this code is _worse_ than the implementation we've had before (longer, likely a bit slower), but it lets us illustrate...

## List Comprehensions

Converting the list of command line arguments from strings to floats is example of a _transformation_: take one list, apply some operatio to each elements (potentially including filtering), and return a new list.

Python has a very succint and efficient construct for writing out such transformations: _list comprehensions_.

These are best understood by looking at a few examples:

In [18]:
lst = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print("Input list: %s" % lst)

squares = [ x**2 for x in lst ]
print("Squares of all elements: %s" % squares)

odd = [ x for x in lst if x % 2 == 1]
print("Odd elements: %s" % odd)

even_squares = [ x**2 for x in lst if x % 2 == 0 ]
print("Squares of even elements: %s" % even_squares)

Input list: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Squares of all elements: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Odd elements: [1, 3, 5, 7, 9]
Squares of even elements: [0, 4, 16, 36, 64, 100]


### Group Work

Let us now rewrite our example (just the `to_floats` function) to use a comprehension instead.

## Implementation v3

In [19]:
show_code_listing("prog11.py")

```python
#!/usr/bin/env python

import sys

def exit_with_msg(msg):
    """ Prints a usage message and exits the program. """
    print("{}\n\nUsage: {} <arg1> <arg2>".format(msg, sys.argv[0]))
    exit(0)

def to_floats(lst):
    """ Returns a list of strings corresponding to a list of floats """
    return [ float(arg) for arg in lst ]

def prod(lst):
    """ Computes and returns the product of a list """
    res = 1.
    for x in lst: res *= x
    return res

#### The main program begins here

try:
    float_list = to_floats( sys.argv[1:] )
    print ( prod( float_list ) )
except ValueError as e:
    exit_with_msg("Error: {}. All arguments must be numbers.".format(e))

```

Note:
* Comprehensions are a concept Python adopted from [functional programming](https://en.wikipedia.org/wiki/Functional_programming) languages.

## Introducing Modules

(paraphrasing from https://docs.python.org/3/tutorial/modules.html):

As your program gets longer, you may want to split it into several files for easier maintenance. You may also want to use a handy function that you’ve written in several programs without copying its definition into each program.

To support this, Python has a way to put definitions in a file and use them in a script or in an interactive session. Such a file is called a **module**; definitions from a module can be **imported** into other modules or into the main program.

For example, if I have a file named `mjutils.py`, {{execute_and_show('cat ./mjutils.py')}}, I could use in another Python program (or in a Jupyter notebook) as:
```python
import mjutils
mjutils.add(3, 4)
```

## Problem 8: Let's Make our Program a Module

The two functions we've written -- `to_float` and `prod` -- seem to be generally useful. For example, if we write a utility that _sums_ the list of numbers given to it on the command line, we'll still want to use something like `to_float` to convert from string to floats.

### Group Work

So let's split off these two functions into their own module, named 'utils'. Then our program will look like:

In [20]:
show_code_listing("prog12.py")

```python
#!/usr/bin/env python

import sys
import utils

def exit_with_msg(msg):
    """ Prints a usage message and exits the program. """
    print("{}\n\nUsage: {} <arg1> <arg2>".format(msg, sys.argv[0]))
    exit(0)

#### The main program begins here

try:
    float_list = utils.to_floats( sys.argv[1:] )
    print ( utils.prod( float_list ) )
except ValueError as e:
    exit_with_msg("Error: {}. All arguments must be numbers.".format(e))

```

## Implementation of the module

In [21]:
show_code_listing("utils.py")

```python
""" A module with useful functions """

def to_floats(lst):
    """ Returns a list of strings corresponding to a list of floats """
    return [ float(arg) for arg in lst ]

def prod(lst):
    """ Computes and returns the product of a list """
    res = 1.
    for x in lst: res *= x
    return res

```

And that's all (well... most :)) there is to it!

### Modules: Enabling Code Re-use

Modules (and collections of modules -- packages) are a way by which Python supports and encourages ***code re-use***. They allow you to create (and share!) collections of useful routines. Virtuall all useful Python routines, as well as the libraries you're familiar with -- numpy, scipy, matplotlib -- are nothing else but Python packages.

In research, your typical workflow will be very similar to what we've done here:
1. Solving a problem (either as a .py script, or in a Jupyter notebook)
1. Noticing that elements of your solution are more broadly useful.
1. Moving them to a module, and changing the code to use the module instead.

## Object Oriented Programming

Finally, let's refresh our memory of [object oriented programming](https://en.wikipedia.org/wiki/Object-oriented_programming), or 'OOP', that was introduced in ASTR 300.

Terminology:
* What is an ***object***? A collection of logically related data and functions that manipulate those data (called ***methods***).
* What is a ***class***? Classes can be thought of as blueprints for creating objects. It specifies which data the object should contain, and what methods will there be to manipulate those data. But a class does not cleare the object itself.

A real-world analogy may be that of a _car_ (a class), and a 2015 Ford Mustang (an _instance_ of a car -- the object).

An example in Python:

In [22]:
class Car:
    """A car for sale by Jeffco Car Dealership.

    Attributes:
        wheels: An integer representing the number of wheels the car has.
        miles: The integral number of miles driven on the car.
        make: The make of the car as a string.
        model: The model of the car as a string.
        year: The integral year the car was built.
    """

    def __init__(self, wheels, miles, make, model, year):
        """Return a new Car object."""
        self.wheels = wheels
        self.miles = miles
        self.make = make
        self.model = model
        self.year = year

    def sale_price(self):
        """Return the sale price for this car as a float amount."""
        return 5000.0 * self.wheels

    def purchase_price(self):
        """Return the price for which we would pay to purchase the car."""
        return 8000 - (.10 * self.miles)

mustang = Car(4, 1000, 'Ford', 'Mustang', 2015)
elantra = Car(4, 25000, 'Hyundai', 'Elantra', 2011)

print(mustang.make, mustang.model, mustang.wheels, mustang.purchase_price(), mustang.sale_price())
print(elantra.make, elantra.model, elantra.wheels, elantra.purchase_price(), elantra.sale_price())

('Ford', 'Mustang', 4, 7900.0, 20000.0)
('Hyundai', 'Elantra', 4, 5500.0, 20000.0)


Things to note:
* We define classes with the 'class' keyword.
* Functions defined within a class are known as `methods`


* We create objects (instances of classes) by "calling" the class as if it was a function. Note how we can create many different objects of a same class.
* We access the data within the object using a `<object>.<field>` notation
* We invoke methods using the same notation.


* All methods defined in the class take a special first argument, by convention named `self`. This is where the data for a particular object instance is stored.
* There's a special method named `__init__`. This method is called when the object is being created; it is called a `constructor` (it constructs the object).


* There are also many other elements to OOP (key being inheritance), that we don't have time to go over today -- check your ASTR 300 notes or the 'Learning Python' book.

## Everything is an object in Python

Objects/classes are **immensly useful** when building organized, reusable, code.

In Python, though you may not think about it, _every_ value is an object. For example, all integers are istances of a class `int`; think of an integer as an "instance of class `int` with an internal state corresponding to the specific integer it represents". Floating point numbers are instances of a class `float`, lists are instances of class `list`, strings are instances of class `str`, all `numpy` arrays you've encoutered are objects, etc, etc. ***Everything is an object in Python.***

## Homework: Write a Calculator module

Problem: write a Python module named `calc` with a class `Calculator` that will emulate a simple calculator capabile of addition, subtraction, multiplication, division, and clearing the state (setting it to zero). It must have methods for all these operations, named `add()`, `sub()`, `mul()`, `div()`, and `clr()`, respectivelly.

Write the `Calculator` class to that it implemets the [_method chaining_](https://en.wikipedia.org/wiki/Method_chaining) programming pattern.

Finally, make sure the class and all of its methods are properly documented with documentation strings ("docstrings") following [proper conventions](https://www.python.org/dev/peps/pep-0257/).

This module (and the class it contains) should work as a drop-in for the following program:

In [23]:
show_code_listing('test_calc.py')

```python
#!/usr/bin/env python

from calc import Calculator

def assertAlmostEqual(a, b):
	""" A function that tests the approximate equality of two floating point numbers. """
	assert round(a-b, 7) == 0, "{} is not equal to {}.".format(a, b)

c1 = Calculator()	# Create an instance of a calculator
c2 = Calculator(50)	# Create another calculator, initialized with 50

# Test individual methods, and that the two instances properly
# track their own state.
c1.add(2);           assertAlmostEqual(c1.result(), 2)
c1.mul(4);           assertAlmostEqual(c1.result(), 8)
c2.add(50);          assertAlmostEqual(c2.result(), 100)
c1.div(8);           assertAlmostEqual(c1.result(), 1)
c1.sub(-3.);         assertAlmostEqual(c1.result(), 4)
c2.div(c1.result()); assertAlmostEqual(c2.result(), 25)

print ("All tests passed! You have a working calculator!")

```

## Solution